## Evaluation
MoA provide scripts to quickly reproduce some of the results, such as evaluations of [AlpacaEval](https://github.com/tatsu-lab/alpaca_eval),
[MT-Bench](https://github.com/lm-sys/FastChat), and [FLASK](https://github.com/kaistAI/FLASK).


### Preparation

In [2]:
# install requirements
!pip install -r requirements.txt -q
!pip install together -q
# 安裝 alpaca_eval
!cd alpaca_eval && pip install -e . -q 
# 安裝 FastChat
!cd FastChat && pip install -e ".[model_worker,llm_judge]" -q

  DEPRECATION: Legacy editable install of alpaca-eval==0.6.2 from file:///app/alpaca_eval (setup.py develop) is deprecated. pip 25.3 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  DEPRECATION: Building 'wavedrom' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wavedrom'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [56]:
import os

# 設定 API Key
os.environ['TOGETHER_API_KEY'] = xx
os.environ['OPENAI_API_KEY'] = xx


### List models of Together.ai

In [3]:
import together
import os

# 設定 API Key
together.api_key = os.environ['TOGETHER_API_KEY']

# 列出所有模型
models = together.Models.list()

# 印出模型 id
for model in models:
    #print(model['id'])

SyntaxError: incomplete input (500674518.py, line 12)

### 小於 10B 的模型整理結果
```text
deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B,
google/gemma-2b-it,
meta-llama/Llama-3-8b-chat-hf,
meta-llama/Llama-3.2-3B-Instruct-Turbo,
meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo,
mistralai/Mistral-7B-Instruct-v0.1,
Qwen/Qwen2.5-7B-Instruct-Turbo,
scb10x/scb10x-llama3-1-typhoon2-8b-instruct
```

### Run AlpacaEval 2 Small models

In [ ]:
!export DEBUG=1

# Change models to SLMs
reference_models="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B,google/gemma-2b-it,meta-llama/Llama-3-8b-chat-hf,Qwen/Qwen2.5-7B-Instruct-Turbo,mistralai/Mistral-7B-Instruct-v0.1"

!python generate_for_alpaca_eval.py \
    --model="Qwen/Qwen2.5-7B-Instruct-Turbo" \
    --output-path="outputs/test2.json" \
    --reference-models="$reference_models" \
    --rounds 1 \
    --num-proc 1
 
#    --reference-models="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B" \
#    --reference-models="$reference_models" \



2025-04-30 01:53:54.980 | INFO     | __main__:main:122 - `reference_models` provided: ['deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', 'google/gemma-2b-it', 'meta-llama/Llama-3-8b-chat-hf', 'Qwen/Qwen2.5-7B-Instruct-Turbo', 'mistralai/Mistral-7B-Instruct-v0.1']. Will generate reference responses on-the-fly.
2025-04-30 01:53:54.980 | INFO     | __main__:main:126 - Start.
Map:   0%|                             | 3/805 [01:24<6:28:04, 29.03s/ examples]

### Use vllm server to do evaluation

In [50]:
!alpaca_eval \
    --model_outputs outputs/test.json \
    --reference_outputs alpaca_eval/results/gpt4_1106_preview/model_outputs.json \
    --annotators_config /app/vllm_annotator.yaml \
    --output_path leaderboard_vllm


INFO:root:Evaluating the Qwen/Qwen2.5-7B-Instruct-Turbo-together outputs.
INFO:root:Creating the annotator from `/app/vllm_annotator.yaml`.
INFO:root:Saving annotations to `/app/annotations_seed0_vllm_annotator.json`.
Annotation chunk:   0%|                                   | 0/7 [00:00<?, ?it/s]INFO:root:Annotating 128 examples with vllm-annotator
INFO:root:Using `openai_completions` on 128 prompts using /app/models/DeepSeek-R1-Distill-Qwen-1.5B.
Annotation chunk:   0%|                                   | 0/7 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/usr/local/bin/alpaca_eval", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/alpaca_eval/main.py", line 611, in main
    fire.Fire(evaluate)
  File "/usr/local/lib/python3.10/dist-packages/fire/core.py", line 135, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/usr/local/lib/python3.10/dist-packages/fire/core.py", line 468, in _Fire


In [52]:
!alpaca_eval \
    --model_outputs outputs/test.json \
    --reference_outputs alpaca_eval/results/gpt4_1106_preview/model_outputs.json \
    --output_path leaderboard_vllm

INFO:root:Evaluating the Qwen/Qwen2.5-7B-Instruct-Turbo-together outputs.
INFO:root:Creating the annotator from `weighted_alpaca_eval_gpt4_turbo`.
INFO:root:Saving annotations to `/usr/local/lib/python3.10/dist-packages/alpaca_eval/evaluators_configs/weighted_alpaca_eval_gpt4_turbo/annotations_seed0_configs.json`.
Annotation chunk:   0%|                                   | 0/7 [00:00<?, ?it/s]INFO:root:Annotating 128 examples with weighted_alpaca_eval_gpt4_turbo
INFO:root:Using `openai_completions` on 128 prompts using gpt-4-1106-preview.
INFO:root:Kwargs to completion: {'model': 'gpt-4-1106-preview', 'temperature': 1, 'logprobs': True, 'top_logprobs': 5, 'is_chat': True}. num_procs=5

prompt_batches:   0%|                                   | 0/128 [00:00<?, ?it/s]WARNING:root:/usr/local/lib/python3.10/client_configs/openai_configs.yaml wasn't found. We are using environment variables to construct the client configs.This is the old and non-recommended way of doing it. Please see `clien